# Introduction

Hello Future Crop friends!

I've appreciated learning about crop stages and growing-degree days. However, I'm left wondering how well we could generalise with unsupervised methods. Can modern machine-learning techniques really not keep up with domain knowledge?

**Can we train a transformer to understand relationships between location, crop-type, time-by-time inputs ($c_t$), and the yield?**

Let's follow the so-called 'scaling laws', just chucking all our data into one big fella and train it to heck.

We feed every data example as inputs (static inputs and time-by-time inputs).
- The network is trained to be generative, recovering masked inputs.
- Then predictions are made by masking only the yield input.

# Data: a first look

The first thing to notice about the dataset, which will help us speed up training, is that there's large variability in the location and crop.
Furthermore, location data fully predicts soil-type and soil nitrogen levels, so we can ignore this data. However, different locations are used more for different crops, so the crop type is still relevant.

In [1]:
# General setup
import os
import gc
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch #for training
from torch import nn
from torch.utils.data import DataLoader, Dataset

#global variables
DATA_DIR = r'/kaggle/input/the-future-crop-challenge'
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Running on {DEVICE}')

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    return

setup_seed(2025) #For reproducibility, set seed to current year

# Check that the kaggle data is all there:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-future-crop-challenge/pr_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tasmax_maize_train.parquet
/kaggle/input/the-future-crop-challenge/sample_submission.csv
/kaggle/input/the-future-crop-challenge/soil_co2_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tas_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/rsds_maize_train.parquet
/kaggle/input/the-future-crop-challenge/tasmin_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tasmax_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/rsds_maize_test.parquet
/kaggle/input/the-future-crop-challenge/soil_co2_maize_test.parquet
/kaggle/input/the-future-crop-challenge/train_solutions_maize.parquet
/kaggle/input/the-future-crop-challenge/pr_maize_test.parquet
/kaggle/input/the-future-crop-challenge/tas_wheat_test.parquet
/kaggle/input/the-future-crop-challenge/tasmax_maize_test.parquet
/kaggle/input/the-future-crop-challenge/pr_maize_train.parquet
/kaggle/input/the-fu

# CropGPT

Below we dig into Pytorch. 
1. Efficient training starts with a dataset class with data pre-moved onto the GPU (when possible).
2. 

## Dataset setup

We will try to train different architectures using pytorch. We spend some time here defining a dataset class.
This will let us train both recurrent networks and transformers later.

In [3]:

class PredDataset(Dataset):
    """
    A pytorch dataset class that transfers all data to GPU, minimising memory transfers during training.
    Expands static data () to the length of the sequence.
    """
    def __init__(self, crop: str, mode: str, data_dir: str, device=device):
        # Read all data files
        self.prepare_inputs(crop, mode, data_dir, device)
    
    def prepare_inputs(self, crop,mode,data_dir,device):
        tasmax = pd.read_parquet(os.path.join(data_dir, f"tasmax_{crop}_{mode}.parquet"))
        tasmin = pd.read_parquet(os.path.join(data_dir, f"tasmin_{crop}_{mode}.parquet"))
        pr = pd.read_parquet(os.path.join(data_dir, f"pr_{crop}_{mode}.parquet"))
        rsds = pd.read_parquet(os.path.join(data_dir, f"rsds_{crop}_{mode}.parquet"))
        soil_co2 = pd.read_parquet(os.path.join(data_dir, f"soil_co2_{crop}_{mode}.parquet"))
        
        # Load yield data if in training mode
        if mode == 'train':
            self.yield_data = pd.read_parquet(os.path.join(data_dir, f"{mode}_solutions_{crop}.parquet"))
            self.yield_data['crop'] = crop
            self.yield_data['crop_id'] = 0 if crop == 'maize' else 1
        else:
            self.yield_data = pd.DataFrame(tasmax['crop'])
            self.yield_data['crop_id'] = 0 if crop == 'maize' else 1
            self.yield_data['yield'] = 0
            
        # Preprocess climate data in bulk (shape: num_samples × 240 × 4)
        climate_data = np.stack([
            tasmax.iloc[:, 5:].values,
            tasmin.iloc[:, 5:].values,
            pr.iloc[:, 5:].values,
            rsds.iloc[:, 5:].values
        ], axis=2).astype(np.float32)
        
        # Preprocess soil data in bulk
        soil_continuous = soil_co2[['lon', 'lat', 'co2']].values.astype(np.float32)
        soil_continuous = np.repeat(soil_continuous[:, np.newaxis, :], 240, axis=1)
        crop_expanded = np.repeat(self.yield_data['crop_id'].values.reshape(-1,1,1),240,axis=1)
        yield_expanded = np.repeat(self.yield_data['yield'].values.reshape(-1,1,1),240,axis=1)
    
        # Combine climate and soil features (shape: num_samples × 240 × 21)
        full_input = np.concatenate([climate_data, soil_continuous, crop_expanded,yield_expanded], axis=2)
        
        # Move entire dataset to device in one operation
        self.inputs = torch.tensor(full_input, device=device,dtype=torch.float32)

        #memory management
        del tasmax, tasmin, pr, rsds, soil_co2, climate_data, soil_continuous, crop_expanded, yield_expanded, full_input

        gc.collect()
        
        return None
        
    def __getitem__(self, index):
        # Return precomputed tensors
        return self.inputs[index]

    def __len__(self):
        return len(self.inputs)

print('Loading training data.,. (this may take a while)')
train_wheat = PredDataset('wheat','train', DATA_DIR)
train_maize = PredDataset('maize','train',DATA_DIR)
print(train_wheat.inputs.shape)

print(train_maize.yield_data)
print(train_wheat.yield_data)
#train_wheat = torch.utils.data.subset

#print('Loading test data... (this may take a while)')
#wheat_test = PredDataset('wheat','test', DATA_DIR, device = 'cpu')

print('Finished loading')



n_train = int(train_maize.inputs.shape[0]*0.8)
n_val = train_maize.inputs.shape[0] - n_train
train_set, val_set = torch.utils.data.random_split(train_maize, [n_train,n_val])

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 512, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size = 512, shuffle = True)

Loading training data.,. (this may take a while)
torch.Size([278747, 240, 9])
        yield   crop  crop_id
ID                           
0       5.595  maize        0
1       5.895  maize        0
2       3.023  maize        0
3       2.071  maize        0
4       2.239  maize        0
...       ...    ...      ...
349714  6.240  maize        0
349715  8.926  maize        0
349716  2.180  maize        0
349717  7.311  maize        0
349718  2.118  maize        0

[349719 rows x 3 columns]
         yield   crop  crop_id
ID                            
1040990  4.775  wheat        1
1040991  4.874  wheat        1
1040992  4.701  wheat        1
1040993  4.848  wheat        1
1040994  5.178  wheat        1
...        ...    ...      ...
1319732  1.418  wheat        1
1319733  1.653  wheat        1
1319734  1.271  wheat        1
1319735  0.469  wheat        1
1319736  0.629  wheat        1

[278747 rows x 3 columns]
Finished loading


## Training transformers



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

class CropTransformer(nn.Module):
    def __init__(self, input_dim, d_model, n_heads, n_layers, max_seq_len, dropout=0.1):
        super().__init__()
        
        # RevIN for Input Features (Climate + Soil). 
        # We assume the last column is Yield and might treat it differently.
        self.revin = RevIN(input_dim - 1) 
        
        self.input_projection = nn.Linear(input_dim, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(1, max_seq_len, d_model))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dim_feedforward=d_model//2, 
            dropout=dropout, batch_first=True, activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, n_layers)
        self.output_projection = nn.Linear(d_model, input_dim)
        self.mask_token = nn.Parameter(torch.randn(1, 1, input_dim))

    def forward(self, x, mask_indices=None):
        batch_size, seq_len, n_feats = x.shape
        
        # --- 1. Apply RevIN (Distribution Shift Handling) ---
        # Strategy: Normalize Inputs (0:-1) to fix distribution shift.
        # Leave Yield (-1) raw to avoid leakage during the 'Yield Prediction' task.
        
        # Split Yield from Features
        x_feats = x[:, :, :-1]
        x_yield = x[:, :, -1:]
        
        # Normalize Features ONLY
        x_feats = self.revin.normalize(x_feats)
        
        # Recombine
        x = torch.cat([x_feats, x_yield], dim=-1)

        # --- 2. Apply Masking ---
        if mask_indices is not None:
            mask_token_expanded = self.mask_token.expand(batch_size, seq_len, -1)
            x = torch.where(mask_indices, mask_token_expanded, x)

        # --- 3. Transformer Block ---
        x = self.input_projection(x) + self.pos_encoding[:, :seq_len, :]
        x = self.transformer(x)
        x = self.output_projection(x)
        
        # --- 4. Denormalize (Optional) ---
        # Since we only normalized inputs, we should technically denormalize the 
        # predicted inputs to calculate loss in the original space.
        
        pred_feats = x[:, :, :-1]
        pred_yield = x[:, :, -1:]
        
        pred_feats = self.revin.denormalize(pred_feats)
        
        # Return recombined full prediction
        return torch.cat([pred_feats, pred_yield], dim=-1)

class RevIN(nn.Module):
    def __init__(self, num_features, eps=1e-5, affine=True):
        """
        Reversible Instance Normalization.
        Normalizes the input over the time dimension to handle distribution shift.
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        if self.affine:
            self._init_params()

    def _init_params(self):
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = 1  # Calculate stats over Sequence Length (dim 1)
        self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()
        return self.mean, self.stdev

    def normalize(self, x):
        mean, stdev = self._get_statistics(x)
        x = (x - mean) / stdev
        if self.affine:
            x = x * self.affine_weight + self.affine_bias
        return x

    def denormalize(self, x):
        if self.affine:
            x = (x - self.affine_bias) / (self.affine_weight + self.eps)
        x = x * self.stdev + self.mean
        return x

def create_dual_objective_mask(inputs, yield_idx=-1, block_size=20, random_prob=0.15):
    """
    Splits the batch into two objectives:
    1. Yield Prediction (First half of batch)
    2. Structure Learning (Second half of batch: Random Index OR Block Masking)
    """
    batch_size, seq_len, n_feats = inputs.shape
    mask = torch.zeros_like(inputs, dtype=torch.bool)
    split_idx = batch_size // 2
    mask[:split_idx, :, yield_idx] = True
    use_block = torch.rand(batch_size - split_idx) > 0.5
    for i in range(split_idx, batch_size):
        is_block_mode = use_block[i - split_idx]
        if is_block_mode:
            start = torch.randint(0, seq_len - block_size, (1,)).item()
            mask[i, start:start+block_size, :] = True
        else:
            rand_time_mask = torch.rand(seq_len) < random_prob
            mask[i, rand_time_mask, :] = True
    return mask, split_idx

def train_one_epoch(model, dataloader, optimizer, scheduler, device, yield_idx=-1):
    model.train()
    epoch_yield_losses = []
    epoch_struct_losses = []
    
    pbar = tqdm(dataloader, desc="Training")
    for inputs in pbar:
        mask, split_idx = create_dual_objective_mask(inputs, yield_idx=yield_idx)
        
        optimizer.zero_grad()
        outputs = model(inputs, mask_indices=mask)
        
        # 1. Yield Loss
        yield_preds = outputs[:split_idx]
        yield_targets = inputs[:split_idx]
        yield_mask = mask[:split_idx]
        
        if yield_mask.any():
            loss_yield = nn.functional.mse_loss(yield_preds[yield_mask], yield_targets[yield_mask])
        else:
            loss_yield = torch.tensor(0.0, device=device, requires_grad=True)
        
        # 2. Structure Loss
        struct_preds = outputs[split_idx:]
        struct_targets = inputs[split_idx:]
        struct_mask = mask[split_idx:]
        
        if struct_mask.any():
            loss_struct = 1e-3*nn.functional.mse_loss(struct_preds[struct_mask], struct_targets[struct_mask])
        else:
            loss_struct = torch.tensor(0.0, device=device, requires_grad=True)
        
        total_loss = loss_yield + loss_struct
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        
        epoch_yield_losses.append(loss_yield.item())
        epoch_struct_losses.append(loss_struct.item())
        
        pbar.set_postfix({'L_Yield': f"{loss_yield.item():.4f}", 'L_Struct': f"{loss_struct.item():.4f}"})
        
    return epoch_yield_losses, epoch_struct_losses

def plot_training_history(yield_history, struct_history):
    plt.figure(figsize=(12, 5))
    def smooth(scalars, weight=0.9):
        if not scalars: return []
        last = scalars[0]
        smoothed = []
        for point in scalars:
            smoothed_val = last * weight + (1 - weight) * point
            smoothed.append(smoothed_val)
            last = smoothed_val
        return smoothed

    plt.plot(yield_history, alpha=0.3, color='blue')
    plt.plot(struct_history, alpha=0.3, color='orange')
    plt.plot(smooth(yield_history), label='Yield Prediction Loss', color='blue', linewidth=2)
    plt.plot(smooth(struct_history), label='Structure Reconstruction Loss', color='orange', linewidth=2)
    plt.title("Loss Trends per Iteration")
    plt.xlabel("Training Iterations (Batches)")
    plt.ylabel("MSE Loss")
    plt.legend()
    plt.grid(True, which='both', alpha=0.3)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Hyperparameters
    BATCH_SIZE = 512
    SEQ_LEN = 240
    N_FEATURES = 8
    D_MODEL = 32
    N_HEADS = 4
    N_LAYERS = 2
    EPOCHS = 10
    LR = 1e-6
    max_LR = 1e-2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # we do a bit of dataset splitting to evaluate how well the training is going
    n_train = int(len(ds_wheat.inputs)*0.8)
    n_val = len(ds_wheat.inputs)-n_train
    
    train_wheat = torch.utils.data.Subset(ds_wheat, range(n_train))
    val_wheat = torch.utils.data.Subset(ds_wheat,range(n_train,n_train+n_val))
    train_loader = DataLoader(train_wheat, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(train_wheat, batch_size=BATCH_SIZE, shuffle=False)
    
    model = CropTransformer(N_FEATURES, D_MODEL, N_HEADS, N_LAYERS, SEQ_LEN).to(device, dtype=torch.float32)
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_LR, 
                                              steps_per_epoch=len(train_loader), 
                                              epochs=EPOCHS)

    all_yield_losses = []
    all_struct_losses = []

    print("Starting Training with Split Batch Strategy...")
    for epoch in range(EPOCHS):
        model.train()
        y_losses, s_losses = train_one_epoch(model, train_loader, 
                                             optimizer, scheduler, device)
        all_yield_losses.extend(y_losses)
        all_struct_losses.extend(s_losses)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1} Avg | Yield: {np.mean(y_losses):.4f} | Struct: {np.mean(s_losses):.4f}")
            val_losses = [] #compute validation loss
            model.eval()
            with torch.no_grad():
                for val_inputs in val_loader:
                    yield_mask = torch.zeros_like(val_inputs, dtype = torch.bool)
                    yield_mask[:,:,-1] = True
                    val_outputs = model(val_inputs, yield_mask)
                    val_loss = nn.functional.mse_loss(val_outputs[yield_mask], 
                                                      val_inputs[:,:,-1].flatten())
                    val_losses.append(val_loss.item())
            print(f'Validation loss:{np.mean(val_losses)}')
    plot_training_history(all_yield_losses, all_struct_losses)

# Submission




In [ ]:
#submission

submission_csv = pd.read_csv('/kaggle/input/the-future-crop-challenge/sample_submission.csv')


data =  PredDataset('maize','test',DATA_DIR)
len(data)


In [ ]:
submission_csv[:553878]

In [ ]:
553878 + 691271